In [2]:
# import libraries
import pandas as pd


In [3]:
# read the data from the csv file located inside datasets folder
df = pd.read_csv('datasets/data.csv', encoding = "ISO-8859-1", low_memory=False)

In [4]:
df.head()

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,150,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01


In [5]:
# check the shape of the dataframe
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns in the dataframe")

There are 435742 rows and 13 columns in the dataframe


In [6]:
# check the basic info of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435742 entries, 0 to 435741
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   stn_code                     291665 non-null  object 
 1   sampling_date                435739 non-null  object 
 2   state                        435742 non-null  object 
 3   location                     435739 non-null  object 
 4   agency                       286261 non-null  object 
 5   type                         430349 non-null  object 
 6   so2                          401096 non-null  float64
 7   no2                          419509 non-null  float64
 8   rspm                         395520 non-null  float64
 9   spm                          198355 non-null  float64
 10  location_monitoring_station  408251 non-null  object 
 11  pm2_5                        9314 non-null    float64
 12  date                         435735 non-null  object 
dtyp

In [7]:
# check the type of data for each column
df.dtypes

stn_code                        object
sampling_date                   object
state                           object
location                        object
agency                          object
type                            object
so2                            float64
no2                            float64
rspm                           float64
spm                            float64
location_monitoring_station     object
pm2_5                          float64
date                            object
dtype: object

In [8]:
# check for null values in descending order
df.isnull().sum().sort_values(ascending=False)

pm2_5                          426428
spm                            237387
agency                         149481
stn_code                       144077
rspm                            40222
so2                             34646
location_monitoring_station     27491
no2                             16233
type                             5393
date                                7
sampling_date                       3
location                            3
state                               0
dtype: int64

In [9]:
# for the float64 columns, fill the null values with the mean of the column
# make a list of the float64 columns
float64_cols = df.select_dtypes(include=['float64']).columns.tolist()
# find the mean and median of each column
mean = df[float64_cols].mean()
#print(mean)
median = df[float64_cols].median()
#print(median)

# fill the null values with the median of the column
df[float64_cols] = df[float64_cols].fillna(median)

# check for null values in descending order
df.isnull().sum().sort_values(ascending=False)

agency                         149481
stn_code                       144077
location_monitoring_station     27491
type                             5393
date                                7
sampling_date                       3
location                            3
state                               0
so2                                 0
no2                                 0
rspm                                0
spm                                 0
pm2_5                               0
dtype: int64

In [10]:
# check for unique values in the dataframe
df.nunique()

stn_code                        745
sampling_date                  5485
state                            37
location                        304
agency                           64
type                             10
so2                            4197
no2                            6864
rspm                           6065
spm                            6668
location_monitoring_station     991
pm2_5                           433
date                           5067
dtype: int64

In [11]:
# check for duplicate rows in the dataframe
print(df.duplicated().sum())

# drop the duplicate rows
df.drop_duplicates(inplace=True)

# check rows and columns after dropping the duplicates
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns in the dataframe")

# are the unique values in the datraframe got affected?
df.nunique()

# they are same as before, so we can proceed

674
There are 435068 rows and 13 columns in the dataframe


stn_code                        745
sampling_date                  5485
state                            37
location                        304
agency                           64
type                             10
so2                            4197
no2                            6864
rspm                           6065
spm                            6668
location_monitoring_station     991
pm2_5                           433
date                           5067
dtype: int64

In [12]:
# drop the columns which are not required
# create a list of columns to be dropped
cols_to_drop = ['agency', 'stn_code', 'date', 'sampling_date', 'location_monitoring_station']

# drop the columns
df.drop(columns=cols_to_drop, inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 435068 entries, 0 to 435741
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   state     435068 non-null  object 
 1   location  435065 non-null  object 
 2   type      429711 non-null  object 
 3   so2       435068 non-null  float64
 4   no2       435068 non-null  float64
 5   rspm      435068 non-null  float64
 6   spm       435068 non-null  float64
 7   pm2_5     435068 non-null  float64
dtypes: float64(5), object(3)
memory usage: 29.9+ MB


In [14]:
# calculate the so2 individual index (si)
def cal_si(so2):
    si = 0
    if (so2 <= 40):
        si = so2 * 50 / 40
    elif (so2 <= 80):
        si = 50 + (so2 - 40) * 50 / 40
    elif (so2 <= 380):
        si = 100 + (so2 - 80) * 100 / 300
    elif (so2 <= 800):
        si = 200 + (so2 - 380) * 100 / 420
    elif (so2 <= 1600):
        si = 300 + (so2 - 800) * 100 / 800
    else:
        si = 400 + (so2 - 1600) * 100 / 800
    return si

# apply the function to calculate si
df['si'] = df['so2'].apply(cal_si)

In [15]:
# calculate the no2 individual index (ni)
def cal_ni(no2):
    ni = 0
    if (no2 <= 40):
        ni = no2 * 50 / 40
    elif (no2 <= 80):
        ni = 50 + (no2 - 40) * 50 / 40
    elif (no2 <= 180):
        ni = 100 + (no2 - 80) * 100 / 100
    elif (no2 <= 280):
        ni = 200 + (no2 - 180) * 100 / 100
    elif (no2 <= 400):
        ni = 300 + (no2 - 280) * 100 / 120
    else:
        ni = 400 + (no2 - 400) * 100 / 120
    return ni

# apply the function to calculate ni
df['ni'] = df['no2'].apply(cal_ni)

In [16]:
# calculate the rspm individual index (rpi)
def cal_rpi(rspm):
    rpi = 0
    if (rspm <= 30):
        rpi = rspm * 50 / 30
    elif (rspm <= 60):
        rpi = 50 + (rspm - 30) * 50 / 30
    elif (rspm <= 90):
        rpi = 100 + (rspm - 60) * 100 / 30
    elif (rspm <= 120):
        rpi = 200 + (rspm - 90) * 100 / 30
    elif (rspm <= 250):
        rpi = 300 + (rspm - 120) * 100 / 130
    else:
        rpi = 400 + (rspm - 250) * 100 / 130
    return rpi

# apply the function to calculate rpi
df['rpi'] = df['rspm'].apply(cal_rpi)

In [17]:
# calculate the spm individual index (spi)
def cal_spi(spm):
    spi = 0
    if (spm <= 50):
        spi = spm
    elif (spm <= 100):
        spi = spm
    elif (spm <= 250):
        spi = 100 + (spm - 100) * 100 / 150
    elif (spm <= 350):
        spi = 200 + (spm - 250)
    elif (spm <= 450):
        spi = 300 + (spm - 350) * 100 / 100
    else:
        spi = 400 + (spm - 450) * 100 / 100
    return spi

# apply the function to calculate spi
df['spi'] = df['spm'].apply(cal_spi)

In [18]:
# calculate the aqi index
def cal_aqi(si, ni, rpi, spi):
    aqi = 0
    if (si > ni and si > rpi and si > spi):
        aqi = si
    elif (ni > si and ni > rpi and ni > spi):
        aqi = ni
    elif (rpi > si and rpi > ni and rpi > spi):
        aqi = rpi
    else:
        aqi = spi
    return aqi

# apply the function to calculate aqi
df['AQI'] = df.apply(lambda x: cal_aqi(x['si'], x['ni'], x['rpi'], x['spi']), axis=1)


In [19]:
# find all the unique states in df
df['state'].unique()

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar',
       'Chandigarh', 'Chhattisgarh', 'Dadra & Nagar Haveli',
       'Daman & Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka',
       'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya',
       'Mizoram', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab',
       'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Uttar Pradesh',
       'Uttarakhand', 'Uttaranchal', 'West Bengal',
       'andaman-and-nicobar-islands', 'Lakshadweep', 'Tripura'],
      dtype=object)

In [20]:
# locate the rows where state is West Bengal
df.loc[df['state'] == 'West Bengal'].location.unique()

array(['Haldia', 'Howrah', 'Calcutta', 'Durgapur', 'Asansol', 'Kolkata',
       'Durgapur (WB)', 'Baruipur', 'Barrackpore', 'Raniganj', 'Sankrail',
       'South Suburban', 'DANKUNI', 'HALDIA', 'Kalyani', 'MALDAH',
       'SILIGURI', 'ULUBERIA'], dtype=object)

# Machine Learning
## use ML to prdict AQI of a city

In [21]:
# create test and train dataframes
test = df.loc[df['state'] == 'West Bengal']
train = df.loc[df['state'] != 'West Bengal']

# check the shape of the dataframes
print(f"Shape of train dataframe: {train.shape}")
print(f"Shape of test dataframe: {test.shape}")



Shape of train dataframe: (412615, 13)
Shape of test dataframe: (22453, 13)


In [23]:
# use lazy predict to find the best model for the dataset
# import the required libraries
import 

ModuleNotFoundError: No module named 'lazypredict'